# Classifying Air Jordan Sneakers
## Sampson Mao

In [1]:
%reload_ext lab_black

In [2]:
import os

import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.config.list_physical_devices("GPU")

In [4]:
DATA_DIR = "C:\\Users\\Sampson\\Desktop\\jordans_classifier\\data"

The images appear to be scraped from the web. Upon further inspection, the "corrupted" data weere html files. Here, I remove those files while making sure the files that are actually images have the right color mode.

In [ ]:
for path, subdirs, files in os.walk(DATA_DIR):
    for filename in files:
        img_path = os.path.join(path, filename)
        try:
            img = Image.open(img_path)
            if img.format not in ["BMP", "GIF", "JPEG", "PNG"]:
                img.convert("RGB").save(img_path)
            elif (img.format == "PNG") and (img.mode != "RGBA"):
                img.convert("RGBA").save(img_path.replace(".jpg", ".png"))
                os.remove(img_path)
        except:
            os.renames(img_path, img_path.replace("data", "corrupted"))

While we do have a sizable number of images for each kind of shoe, many of the images do not provide much variety. For example, AJ1s seem to have the same images cropped or zoomed differently. Ideally we would want to remove duplicates but as an initial exploration, we will try data augmentation.

In [6]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2,
)

In [7]:
train_ds = datagen.flow_from_directory(
    DATA_DIR,
    subset="training",
    target_size=(256, 256),
    class_mode="sparse",
)
val_ds = datagen.flow_from_directory(
    DATA_DIR,
    subset="validation",
    target_size=(256, 256),
    class_mode="sparse",
)

Found 7504 images belonging to 21 classes.
Found 1867 images belonging to 21 classes.


Here I create a basic Sequential model with 2 convolutional layers.

In [8]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(
            filters=16, kernel_size=(3, 3), activation="relu", input_shape=(256, 256, 3)
        ),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(21, activation="softmax"),
    ]
)

In [9]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100
235/235 [==============================] - 120s 500ms/step - loss: 4.4520 - accuracy: 0.0837 - val_loss: 2.8521 - val_accuracy: 0.1002
Epoch 2/100
235/235 [==============================] - 115s 490ms/step - loss: 2.8106 - accuracy: 0.1066 - val_loss: 2.8499 - val_accuracy: 0.0734
Epoch 3/100
235/235 [==============================] - 113s 480ms/step - loss: 2.7446 - accuracy: 0.1305 - val_loss: 2.9064 - val_accuracy: 0.0927
Epoch 4/100
235/235 [==============================] - 126s 536ms/step - loss: 2.7225 - accuracy: 0.1390 - val_loss: 2.9027 - val_accuracy: 0.0889
Epoch 5/100
235/235 [==============================] - 127s 539ms/step - loss: 2.6621 - accuracy: 0.1680 - val_loss: 2.8802 - val_accuracy: 0.1028
Epoch 6/100
235/235 [==============================] - 120s 512ms/step - loss: 2.6286 - accuracy: 0.1810 - val_loss: 2.9029 - val_accuracy: 0.1296
Epoch 7/100
235/235 [==============================] - 113s 479ms/step - loss: 2.5713 - accuracy: 0.1990 - val_loss: 2